In [1]:
import json
import gensim
import urllib2
import pyLDAvis.gensim
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from gensim import corpora, models, similarities


stop_words = set(stopwords.words('english'))
stop_words.update(['.',  ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '#', '@', '##########', '-', ')-', '-------------------------------------------------', '!!']) # remove if you need punctuation

def get_instagram_caption_terms(hashtag):
    search_url = 'https://www.instagram.com/explore/tags/' + hashtag + '/?__a=1'
    contents = urllib2.urlopen(search_url).read()
    results = json.loads(contents)
    edges = results['graphql']['hashtag']['edge_hashtag_to_media']['edges']
    for edge in edges:
        captions = edge['node']['edge_media_to_caption']['edges']
        for caption in captions:
            text = caption['node']['text']
            words = [i.lower() for i in wordpunct_tokenize(text.encode('ascii', 'ignore')) if i.lower() not in stop_words]
            all_tokens = ' '.join(words)
    texts = words
    # remove words that appear only once
    tokens_once = set(words for words in set(all_tokens) if all_tokens.count(words) == 1)
    texts = [[words for words in texts if words not in tokens_once]
             for words in all_tokens]
    return texts
    

texts = get_instagram_caption_terms('acroyoga') + \
        get_instagram_caption_terms('pointe') + \
        get_instagram_caption_terms('watercolor')


#Setup gensim dictionary
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda = gensim.models.LdaModel(corpus, id2word=dictionary, alpha='auto', num_topics=10)
for i in lda.show_topics():
    print (i)

topic_vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

(0, u'0.049*"artist" + 0.027*"color" + 0.026*"watercolour" + 0.025*"art" + 0.025*"sketch" + 0.025*"artistic" + 0.025*"drawing" + 0.025*"art_community" + 0.024*"toptags" + 0.024*"myart"')
(1, u'0.061*"artist" + 0.034*"art" + 0.027*"painting" + 0.026*"ink" + 0.026*"instaartoftheday" + 0.026*"artistsofinstagram" + 0.026*"creative" + 0.026*"artstagram" + 0.026*"illustration" + 0.026*"artoftheday"')
(2, u'0.045*"flexible" + 0.030*"gym" + 0.030*"..." + 0.030*"dancers" + 0.029*"fun" + 0.029*"today" + 0.028*"acroyoga" + 0.028*"friends" + 0.028*"acrobatics" + 0.027*"gymnasticsshoutouts"')
(3, u'0.050*"artist" + 0.034*"color" + 0.033*"watercolour" + 0.032*"myart" + 0.031*"toptags" + 0.031*"instaartoftheday" + 0.030*"artlovers" + 0.030*"artistsoninstagram" + 0.030*"drawing" + 0.029*"ink"')
(4, u'0.036*"dancer" + 0.026*"dancers" + 0.026*"flexible" + 0.024*"dance" + 0.024*"ballet" + 0.023*"artist" + 0.019*"bad" + 0.017*"someone" + 0.016*"capezio" + 0.016*"dancing"')
(5, u'0.054*"artist" + 0.034*"co

/anaconda2/envs/anaconda-env/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [2]:
pyLDAvis.display(topic_vis)